# Analyze data using ilastik
The notebook shows how load images contained in a dataset
from OMERO as numpy arrays and analyze them in ilastik.
It assumes that ilastik project is linked to the dataset.
Binary data are read from S3.
The order might need to be adjusted depending on the ilastik project.

### Insert required packages

In [1]:
import numpy
import os
import itertools
import zarr
import dask.array as da
import s3fs

import tempfile

import omero.clients
from omero.gateway import BlitzGateway
from getpass import getpass
from collections import OrderedDict

from collections import OrderedDict
import ilastik_main
from ilastik.applets.dataSelection import DatasetInfo
from ilastik.applets.dataSelection.opDataSelection import PreloadedArrayDatasetInfo
import ipywidgets as widgets

### Create a connection to an OMERO server

In [2]:
HOST = 'wss://outreach.openmicroscopy.org/omero-ws'
conn = BlitzGateway(input("Username: "),
                    getpass("OMERO Password: "),
                    host=HOST, secure=True)
conn.connect()

Username: trainer-1
OMERO Password: ········


True

### Enter the dataset ID

In [3]:
dataset_id = 6161

### Load the ilastik projects linked to the dataset

In [4]:
def load_model(dataset_id, path):
    dataset = conn.getObject("Dataset", dataset_id)
    # Go through all the annotations on the Dataset
    options = []
    for ann in dataset.listAnnotations():
        if isinstance(ann, omero.gateway.FileAnnotationWrapper):
            name = ann.getFile().getName()
            # Select the ilatisk project TODO: use namespace
            if name.endswith(".ilp"):
                file_path = os.path.join(path, name)
                options.append((name, file_path))
                with open(str(file_path), 'wb') as f:
                    for chunk in ann.getFileInChunks():
                        f.write(chunk)
    return widgets.Dropdown(options=options, disabled=False)

### Helper function: load an Image as 5D-numpy array: order TZYXC

In [5]:
def load_from_s3(image, resolution='0'):
    cache_size_mb = 2048
    # ID to be changed
    id = 6001240
    cfg = {
        'anon': True,
        'client_kwargs': {
            'endpoint_url': 'https://minio-dev.openmicroscopy.org/',
        },
        'root': 'idr/zarr/v0.1/%s.zarr/%s/' % (id, resolution)
    }
    s3 = s3fs.S3FileSystem(
        anon=cfg['anon'],
        client_kwargs=cfg['client_kwargs'],
    )
    store = s3fs.S3Map(root=cfg['root'], s3=s3, check=False)
    cached_store = zarr.LRUStoreCache(store, max_size=(cache_size_mb * 2**20))
    # data.shape is (t, c, z, y, x) by convention
    data = da.from_zarr(cached_store)
    values = data[:]
    values = values.swapaxes(1, 2).swapaxes(2, 3).swapaxes(3, 4)
    return numpy.asarray(values)

### Create a temporary directory where to download the ilastik project

In [6]:
path = tempfile.mkdtemp()
if not os.path.exists(path):
    os.makedirs(path)

### Select the ilastik project to use.

In [7]:
model_selection = load_model(dataset_id, path)
display(model_selection)

Dropdown(options=(('pixel-class-133.ilp', '/tmp/tmpg_7yi6sx/pixel-class-133.ilp'),), value='/tmp/tmpg_7yi6sx/p…

### Load each image as an 5D-numpy array and analyze.
Save the probabilities as Zarr zip and link it to the image.

In [8]:
# Load the model linked to the dataset
model_file = model_selection.value


images = conn.getObjects('Image', opts={'dataset': dataset_id})

# Create a new dataset where to upload the generated images
dataset_obj = omero.model.DatasetI()
v = "ilastik_probabilities_from_dataset_%s" % dataset_id
dataset_obj.setName(omero.rtypes.rstring(v))
v = "ilatisk results probabilities from Dataset:%s" % dataset_id
dataset_obj.setDescription(omero.rtypes.rstring(v))
dataset_obj = conn.getUpdateService().saveAndReturnObject(dataset_obj)

# Prepare ilastik
os.environ["LAZYFLOW_THREADS"] = "2"
os.environ["LAZYFLOW_TOTAL_RAM_MB"] = "2000"
args = ilastik_main.parse_args([])
args.headless = True
args.project = model_file
shell = ilastik_main.main(args)

images = itertools.islice(images, 2)
for image in images:
    filename, file_extension = os.path.splitext(image.getName())
    input_data = load_from_s3(image)

    # run ilastik headless
    print('running ilastik using %s and %s' % (model_file, image.getName()))
    role_data_dict = OrderedDict(
    [
        (
            "Raw Data",
            [
                PreloadedArrayDatasetInfo(preloaded_array=input_data)
            ],
        )
    ])

    predictions = shell.workflow.batchProcessingApplet.run_export(role_data_dict, export_to_array=True)
    # Save the probabilities file to the image
    print("Saving Probabilities as an Image in OMERO")
    namespace = "ilastik.zarr.demo"
    name = filename + "_Probabilities_zarr.zip"
    desc = "ilastik probabilities from Image:%s" % image.getId()
    for data in predictions:
        file_path = os.path.join(path, name)
        with zarr.ZipStore(file_path, mode='w') as store:
            zarr.array(data, store=store, dtype='int16', compressor=zarr.Blosc(cname='zstd'))
        ann = conn.createFileAnnfromLocalFile(file_path, mimetype="application/zip", ns=namespace, desc=desc)
        image.linkAnnotation(ann)

print("done")

INFO ilastik_main: Starting ilastik from "/srv/conda/envs/notebook".


INFO:ilastik_main:Starting ilastik from "/srv/conda/envs/notebook".


Starting ilastik from "/srv/conda/envs/notebook".
INFO ilastik_main: Resetting lazyflow thread pool with 2 threads.


INFO:ilastik_main:Resetting lazyflow thread pool with 2 threads.


INFO ilastik_main: Configuring lazyflow RAM limit to 2.0GiB


...

INFO lazyflow.utility.memory: Available memory set to 2.0GiB


INFO:lazyflow.utility.memory:Available memory set to 2.0GiB
WARNING 2020-04-22 10:06:37,909 memory 61 140033343121216 User specified memory exceeds memory physically available. Please check theconfiguration.
WARNING 2020-04-22 10:06:38,808 __init__ 61 140033343121216 Failed to automatic tracking workflow; check dependencies: libemon.so.1.3.1: cannot open shared object file: No such file or directory
WARNING 2020-04-22 10:06:38,813 __init__ 61 140033343121216 Failed to import automatic tracking workflow (conservation tracking). For this workflow, see the installationinstructions on our website ilastik.org; check dependencies: libemon.so.1.3.1: cannot open shared object file: No such file or directory
WARNING 2020-04-22 10:06:38,822 __init__ 61 140033343121216 Failed to import structured learning tracking workflow. For this workflow, see the installationinstructions on our website ilastik.org; check dependencies: libemon.so.1.3.1: cannot open shared object file: No such file or directory

INFO ilastik.shell.projectManager: Opening Project: /tmp/tmpg_7yi6sx/pixel-class-133.ilp


INFO:ilastik.shell.projectManager:Opening Project: /tmp/tmpg_7yi6sx/pixel-class-133.ilp
WARNING projectManager.py(158): H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
WARNING projectManager.py(168): H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
WARNING dataSelectionSerializer.py(385): H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
WARNING featureSelectionSerializer.py(91): H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
WARNING featureSelectionSerializer.py(96): H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
WARNING featureSelectionSerializer.py(102): H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
WARNING featureSelectionSerializer.py(122): H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
WARNING opSimpleBlockedArrayCache.py(43): Fut

HTTPClientError: An HTTP Client raised and unhandled exception: maximum recursion depth exceeded while calling a Python object

### Close the connection to the OMERO server

In [10]:
conn.close()

### License
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.